In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pickle,os,time
import trako as TKO

In [3]:
SUBJECT='/'
BLACKLIST = 'tracts_commissural'
DATAFOLDER = '/home/haehn/Dropbox/TRAKODATA/AnatomicalTracts/'
# DATAFOLDER = '/home/haehn/Dropbox/TKOTEST/'
DATAFOLDER_TKO = DATAFOLDER[:-1]+'_TKO/'
DATAFOLDER_RESTORED = DATAFOLDER[:-1]+'_RESTORED/'
STATSFOLDER = DATAFOLDER[:-1]+'_STATS/'
DATAFOLDER += SUBJECT
DATAFOLDER_TKO += SUBJECT
DATAFOLDER_RESTORED += SUBJECT
STATSFILE = STATSFOLDER+SUBJECT[:-1]+'.p'

originalsize = []
compressedsize = []
number_of_streamlines = []
c_time = []
d_time = []


In [23]:
input_size = []
for f in os.listdir(DATAFOLDER):

        if f.endswith('.vtp') and not 'restored' in f:
            input = os.path.join(DATAFOLDER, f)
            input_size.append(os.path.getsize(input))

In [24]:
compressed_size = []
for f in os.listdir(DATAFOLDER):

        if f.endswith('_compressed_configqbi10_binary.tko.glb'):
            input = os.path.join(DATAFOLDER, f)
            compressed_size.append(os.path.getsize(input))

In [25]:
np.sum(input_size)

32882748

In [26]:
np.sum(compressed_size)

13428576

In [8]:
THRESHOLD = 10
scalarstats = {
    
}
propertystats = {
    
}
counter = 0
firstrun = True
t0 = time.time()
for f in os.listdir(DATAFOLDER):

        if f.endswith('.vtp') and not 'restored' in f:
            input = os.path.join(DATAFOLDER, f)
            
            
            
#             compressed = input.replace('.vtp','.tko')
            restored = input.replace('.vtp', '.vtp_restored_configqbi10.vtp')
            
            i_poly = TKO.Util.loadvtp(input)
            i_scalars = i_poly['scalars']
            i_nscalars = i_poly['scalar_names']
            i_properties = i_poly['properties']
            i_nproperties = i_poly['property_names']
            
            r_poly = TKO.Util.loadvtp(restored)
            r_scalars = r_poly['scalars']
            r_properties = r_poly['properties']
            
#             print(i_poly['points'])
#             print(r_poly['points'])
#             print(i_scalars)
#             print(r_scalars)

            if len(i_scalars)==0 or len(r_scalars) == 0:
                print('skipped', name)
                continue
                
#             if (np.isnan(np.any(i_scalars))):
#                 print('NaN', name, i_scalars)
#                 continue
                
#             if (np.isnan(np.any(r_scalars))):
#                 print('NaN2', name, i_scalars)
#                 continue
            
            if firstrun:
                for i,s in enumerate(i_nscalars):
                    scalarstats[s] = [0,0,0,0,np.inf,-np.inf]
#                     print ('reset')
                for i,p in enumerate(i_nproperties):
                    propertystats[p] = [0,0,0,0,np.inf,-np.inf]
                
                firstrun = False
    
    
            
            for i,s in enumerate(i_scalars):
                c_name = i_nscalars[i]
            
                stats = TKO.Util.error(i_scalars[i], r_scalars[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

                scalarstats[c_name][0] += stats[0][0]
                scalarstats[c_name][1] += stats[0][1]
                scalarstats[c_name][2] += stats[0][2]
                scalarstats[c_name][3] += stats[0][3]
                scalarstats[c_name][4] = min(np.min(i_scalars[i]),scalarstats[c_name][4] )
                scalarstats[c_name][5] = max(np.max(i_scalars[i]),scalarstats[c_name][5] )

            for i,s in enumerate(i_properties):
                c_name = i_nproperties[i]
            
                stats = TKO.Util.error(i_properties[i], r_properties[i])
#                 print(stats)

#                 if (np.isnan(stats[0][0])):
#                     print(input, counter, r_scalars)

                propertystats[c_name][0] += stats[0][0]
                propertystats[c_name][1] += stats[0][1]
                propertystats[c_name][2] += stats[0][2]
                propertystats[c_name][3] += stats[0][3]
                propertystats[c_name][4] = min(np.min(i_properties[i]),propertystats[c_name][4] )
                propertystats[c_name][5] = max(np.max(i_properties[i]),propertystats[c_name][5] )
                
            counter += 1
        if counter > THRESHOLD:
            break
            
# normalize
for s in scalarstats.keys():
    for i,v in enumerate(scalarstats[s]):
        if i > 3:
            continue
        scalarstats[s][i] /= counter
for s in propertystats.keys():
    for i,v in enumerate(propertystats[s]):
        if i > 3:
            continue
        propertystats[s][i] /= counter
print(time.time()-t0)

4.52129864692688


In [9]:
scalarstats

{'EstimatedUncertainty': [0.0,
  0.2753684303977273,
  0.1347860497507182,
  0.080928696827455,
  0.031622775,
  15233.791],
 'tensor2': [2.344598010689031e-08,
  1.3793757086039643e-07,
  8.731460419333169e-08,
  1.781123095476935e-08,
  -0.0008659273,
  0.0020841828],
 'HemisphereLocataion': [0.0, 0.0, 0.0, 0.0, 1.0, 3.0],
 'cluster_idx': [0.0,
  0.8181818181818182,
  0.24584532274382476,
  0.3614032346871214,
  0,
  39],
 'tensor1': [2.8291385073037582e-08,
  1.7842506462205743e-07,
  1.1212859295607235e-07,
  2.2736418969770966e-08,
  -0.0009526277,
  0.0023900766]}

In [10]:
propertystats

{'EmbeddingCoordinate': [0.00016411313250004738,
  0.0003467026769860902,
  0.00026175899917937136,
  3.721884670746724e-05,
  -4.542453,
  3.0473692],
 'ClusterNumber': [0.0, 1.0, 0.42374512385046914, 0.47632639176233393, 8, 665],
 'EmbeddingColor': [0.0,
  1.3765577839755414,
  0.8775463746838645,
  0.47481806050288583,
  0,
  180],
 'TotalFiberSimilarity': [0.0,
  15.903409090909092,
  8.019401463595303,
  4.754698276519775,
  199220.9,
  920767.2],
 'MeasuredFiberSimilarity': [0.0,
  1.517632468180223e-08,
  7.408974363313932e-09,
  4.543307689659896e-09,
  0.0017869201,
  0.0026580554]}

In [13]:
prec = 10
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],prec),np.round(scalarstats[p][5],prec)
    mean= np.round(scalarstats[p][2],prec),np.round(scalarstats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in scalarstats.keys():
    ranger = np.round(scalarstats[p][4],prec),np.round(scalarstats[p][5],prec)
    mean= np.round(scalarstats[p][2],prec),np.round(scalarstats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

EstimatedUncertainty ($N$, range:  0.031622775-15233.791 ) \\
tensor2 ($N$, range:  -0.0008659273-0.0020841828 ) \\
HemisphereLocataion ($N$, range:  1.0-3.0 ) \\
cluster_idx ($N$, range:  0-39 ) \\
tensor1 ($N$, range:  -0.0009526277-0.0023900766 ) \\
0.1347860498$\pm$0.0809286968 \\
8.73e-08$\pm$1.78e-08 \\
0.0$\pm$0.0 \\
0.2458453227$\pm$0.3614032347 \\
1.121e-07$\pm$2.27e-08 \\


In [14]:
prec = 10
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],prec),np.round(propertystats[p][5],prec)
    mean= np.round(propertystats[p][2],prec),np.round(propertystats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(p, '($N$, range: ',ranger_str, ') \\\\')
for p in propertystats.keys():
    ranger = np.round(propertystats[p][4],prec),np.round(propertystats[p][5],prec)
    mean= np.round(propertystats[p][2],prec),np.round(propertystats[p][3],prec)
    ranger_str = str(str(np.round(ranger[0],prec))+'-'+str(np.round(ranger[1],prec)))
    mean_str = str(mean[0])+'$\pm$'+str(mean[1])
    print(mean_str , '\\\\')

EmbeddingCoordinate ($N$, range:  -4.542453-3.0473692 ) \\
ClusterNumber ($N$, range:  8-665 ) \\
EmbeddingColor ($N$, range:  0-180 ) \\
TotalFiberSimilarity ($N$, range:  199220.9-920767.25 ) \\
MeasuredFiberSimilarity ($N$, range:  0.0017869202-0.0026580554 ) \\
0.000261759$\pm$3.72188e-05 \\
0.4237451239$\pm$0.4763263918 \\
0.8775463747$\pm$0.4748180605 \\
8.0194014636$\pm$4.7546982765 \\
7.4e-09$\pm$4.5e-09 \\


In [28]:
compressedsize = np.sum(compressed_size)
originalsize = np.sum(input_size)

c_ratio = (1-float(compressedsize)/float(originalsize))*100
c_factor = float(originalsize) / float(compressedsize)

In [29]:
np.round(c_ratio,3)

59.162

In [30]:
np.round(c_factor,3)

2.449

In [32]:
np.round(compressedsize/1000000,2)

13.43